In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 CLI
######################################################################## 100.0%#=#=#                                                                         
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [1]:
from langchain_community.llms import Ollama
from IPython.display import Markdown
from pathlib import Path

In [2]:
from crewai import Agent, Task, Crew
from crewai_tools import BaseTool

class PromptingPatientTool(BaseTool):
    name: str = "Patient Prompting"
    description: str = "When more information is needed from the patient, use this tool. "
    "Don't comment on the result afterwards. Do not add anything. Just return the result. "

    def _run(self, question: str) -> str:
        # Your tool's logic here
        return f"{question}"
    
class WriteToFileTool(BaseTool):
    name: str = "File Writer Tool"
    description: str = "Writing information to report.txt. Provide content to be written, and must include a flag to overwrite an existing file (w to rewrite the file (for new patients), a to extend the file). "
    "You must write down the question and answer as for the 'content' parameter in  format like this: Question: [question asked], Answer: [answer to the question]."
    
    def _run(self, filename: str, path: str, content: str, flag: str) -> str:
        # Your tool's logic here
        with open(Path('.') / 'report.txt', flag) as f:
            f.write(content)
            f.write('\n')
            
class ReadFileTool(BaseTool):
    name: str = "File Reader Tool"
    description: str = "Allows the reading of a file"
    
    def _run(self, filename: str, path: str) -> str:
        # Your tool's logic here
        with open(Path('.') / 'report.txt', 'r') as f:
            return f.read()
    
class CommunicateWithExpert(BaseTool):
    name: str = "Communication with a MedicalExpert Tool"
    description: str = "Commun"

/root/.local/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
from crewai_tools import FileReadTool, FileWriterTool

prompt_tool = PromptingPatientTool()
file_read_tool = ReadFileTool()
file_write_tool = WriteToFileTool()

In [23]:
from crewai import Agent, Task, Crew, Process

llm = Ollama(model='llama3.1')
bio_llm = Ollama(model='jsk/bio-mistral')

nurse = Agent(
    role="A hospital nurse",
    goal="Your goal is to gather patient information and symptoms by coming up with a single question for the patient to obtain pertinent information."
    "You can only output either a question or a summary of the information returned by your tools",
    backstory="The AI agent is modeled after a seasoned nurse with extensive experience in primary care." 
    "It has been trained on a wide range of medical conditions and symptoms, allowing it to ask pertinent questions and recognize key details. "
    "The agent’s persona is designed to be empathetic and attentive, reflecting a caring and professional demeanor. "
    "It is equipped with a robust knowledge base to handle diverse medical queries and provide a supportive conversational experience. ",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

doctor = Agent(
    role="A hospital doctor",
    goal="Your goal is to diagnose the patient and identify the disease that they might be suffering from. "
    "You also have to provide the next course of action for the patient. "
    "You also might need to call in a specialist (for example: a dermatologist for skin diseases) if one is available. ",
    backstory="You are a doctor working at a hospital. You have to diagnose patients' diseases given the nurse's reports.",
    verbose = True,
    allow_delegation=False,
    llm=llm
)

manager = Agent(
    role="Control flow manager",
    goal="Given the nurse's task output, your goal is to simply the information to the patient exactly as is. "
    "Output the information if it is a question, send the information in for diagnosis if it is a summary of symptoms. Don't answer the nurse. ",
    backstory="You are an organized manager at a hospital tasked with communicating the right information to a patient. ",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [32]:
diagnose = Task(
    description=("Given the nurse's reported information from the patient: {symptoms}. You must make an assessment of the results. "
                "If more information is needed, make a brief statement and provide additional questions for the patient. "),
    expected_output="You must provide a diagnosiss or potential diagnoses. "
    "You must determine whether more information is needed from the patient and the next course of action. "
    "Tell the nurse whether more information is needed, provide questions for the patient, and consider the task done. "
    "If no specialists are available, provide course of action and treatment plan. ",
    tools=[],
    agent=doctor
)

communicate_back = Task(
    description=("Given the doctor agent's outputs, report it back to the patient in a clear, concise, and understandable manner. "),
    expected_output="A clear explanation of what the patient is suffering from based on the doctor's reports. List the next course of action. "
    "Tell the patient what he would need to get checked for at a real hospital. "
    "If the patient needs to get to the hospital, inform the patient that this is only a virtual hospital and that the patient needs medical care at the hospital",
    agent=nurse
)

diagnose_crew = Crew(
    agents=[doctor, nurse],
    tasks=[diagnose, communicate_back],
    verbose=True
)


class InferenceDiagnosisCrew(BaseTool):
    name: str = "Inference Diagnosis Team Tool"
    description: str = "This tool can ONLY be used when enough information about the patient has been gathered. Do not use this tool otherwise. "
    "The information will ready to be processed by the medical team. "
    "The output is a medical diagnosis. "
    
    def _run(self, symptoms: str) -> str:
        # Your tool's logic here
        return diagnose_crew.kickoff(inputs={'symptoms': symptoms}).raw
    
collect_info = Task(
    description=("Given the patient's message: {message}. Highlight important symptoms. "
                 "You will collect patient information by asking questions. You will output a question for further precision. "
                 "If enough information has been recieved, report the information in a bullet point format to the doctor. "
                "Don't add anything after using the tools. Simply relay the information."),
    expected_output="A single question about the patient's symptoms. In addition, add True if it is a question, False if it isn't at the end. Either that, or a bullet list of all of the patient's symptoms. ",
    tools=[],
    agent=nurse
)

relay = Task(
    description=("Determine if the information is ready to be sent to the diagnosis team. If provided with a question, simply output the information with no modifications and do not answer the question. "
                "ONLY IF provided with a long list of symptoms, use the Inference Diagnosis Crew tool with the symptoms as input."),
    expected_output="Either a question or a diagnosis from the doctor. In addition, True if it is a question, False if it isn't",
    tools=[],
    agent=manager,
)

2024-08-21 20:36:16,923 - 140312225226752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [33]:
prompt_crew = Crew(
    agents=[nurse],
    tasks=[collect_info],
    verbose=True,
)


2024-08-21 20:36:17,140 - 140312225226752 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [34]:
display(Markdown(prompt_crew.kickoff().raw))

 [2024-08-21 20:36:17][DEBUG]: == Working Agent: A hospital nurse
 [2024-08-21 20:36:17][INFO]: == Starting Task: Given the patient's message: {message}. Highlight important symptoms. You will collect patient information by asking questions. You will output a question for further precision. If enough information has been recieved, report the information in a bullet point format to the doctor. Don't add anything after using the tools. Simply relay the information.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: What brings you to the hospital today? Is there anything specific that's been concerning or bothering you recently?

True

> Finished chain.
 [2024-08-21 20:36:18][DEBUG]: == [A hospital nurse] Task output: What brings you to the hospital today? Is there anything specific that's been concerning or bothering you recently?

True




What brings you to the hospital today? Is there anything specific that's been concerning or bothering you recently?

True

In [35]:
import gradio as gr
from gradio import ChatMessage

In [36]:
import random
import torch

def random_response(message, history):
    
    answer = prompt_crew.kickoff(inputs={'message':message}).raw
    if "False" in answer.lower():
        pass
        # answer = diagnose_crew.kickoff(answer).raw
    return answer

gr.ChatInterface(random_response).launch(share=True)

Running on local URL:  http://127.0.0.1:7876
Running on public URL: https://d96713e33a9b9fcf51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 [2024-08-21 20:36:30][DEBUG]: == Working Agent: A hospital nurse
 [2024-08-21 20:36:30][INFO]: == Starting Task: Given the patient's message: i have a headache along with blood vomiting with loss of vision and nausea around 2 weeks ago when i ate spoiled rice. It was very abupt. I also experienced a high fever. Highlight important symptoms. You will collect patient information by asking questions. You will output a question for further precision. If enough information has been recieved, report the information in a bullet point format to the doctor. Don't add anything after using the tools. Simply relay the information.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: When did the high fever start in relation to when you experienced the headache and vomiting, was it before or after eating the spoiled rice? True.

> Finished chain.
 [2024-08-21 20:36:31][DEBUG]: == [A hospital nurse] Task output: When did the high fever start in relation 